<a href="https://colab.research.google.com/github/nitishchauhan002/4-queen-problem-/blob/main/traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install kagglehub --quiet
import kagglehub

In [11]:
path = kagglehub.dataset_download("javiersanchezsoriano/traffic-images-captured-from-uavs")
print("Path to dataset files:", path)


100%|██████████| 32.3G/32.3G [05:04<00:00, 114MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3


In [1]:
import os

In [12]:
from google.colab.patches import cv2_imshow

# New Section

In [14]:
import shutil

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
import cv2


In [17]:
import numpy as np

In [18]:
import matplotlib.pyplot as plt

In [19]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)

In [20]:
image_folder = os.path.join(path, "images")  # 📸 Actual image folder name देखना!
processed_folder = os.path.join(path, "processed_images")
os.makedirs(processed_folder, exist_ok=True)

In [21]:
image_path = os.path.join(image_folder, 'your_image.jpg')

In [22]:
img = preprocess_image(image_path)

In [23]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)

    if img is None:
        print(f"❌ Error: Cannot load {image_path}. Skipping...")
        return None

    # Resize Image
    img = cv2.resize(img, (416, 416))

    # Apply Gaussian Blur
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # Normalize image
    img = img / 255.0

    return img

In [24]:
image_count = 0
if os.path.exists(image_folder):
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)
            processed_img = preprocess_image(image_path)

            if processed_img is not None:
                save_img = (processed_img * 255).astype(np.uint8)
                save_path = os.path.join(processed_folder, filename)
                cv2.imwrite(save_path, save_img)
                image_count += 1
else:
    print(f"❌ Image folder not found at: {image_folder}")

print(f"🎯 Preprocessing Complete! Total {image_count} images saved to: {processed_folder}")

❌ Image folder not found at: /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/images
🎯 Preprocessing Complete! Total 0 images saved to: /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/processed_images


processing complete


In [25]:
!pip install ultralytics --upgrade --quiet
from ultralytics import YOLO

print("✅ YOLOv8 Installed Successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultra

In [26]:
dataset_yaml = """
path: {path}  # Dataset root path
train: processed_images
val: processed_images

names:
  0: car
  1: bike
  2: truck
"""

yaml_path = os.path.join(path, "dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(dataset_yaml.format(path=path))

print("✅ Dataset config file created at:", yaml_path)


✅ Dataset config file created at: /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset.yaml


In [27]:
import os
import random
import shutil

image_dir = os.path.join(path, "dataset")
train_dir = os.path.join(path, "images/train")
val_dir = os.path.join(path, "images/val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

images = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            images.append(os.path.join(root, file))

random.shuffle(images)

split_idx = int(0.8 * len(images))
train_images = images[:split_idx]
val_images = images[split_idx:]

for img in train_images:
    shutil.copy(img, train_dir)

for img in val_images:
    shutil.copy(img, val_dir)

print(f"✅ Split done: {len(train_images)} train and {len(val_images)} val images.")


OSError: [Errno 28] No space left on device: '/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec8/sec8_frame_001091.png' -> '/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/images/val/sec8_frame_001091.png'

In [33]:
# Create Dummy YOLO Labels if not created earlier
label_train = os.path.join(path, "labels/train")
label_val = os.path.join(path, "labels/val")
os.makedirs(label_train, exist_ok=True)
os.makedirs(label_val, exist_ok=True)

# For Train Images
for img in os.listdir(os.path.join(path, "images/train")):
    if img.endswith(".jpg") or img.endswith(".png"):
        label_path = os.path.join(label_train, img.replace('.jpg', '.txt').replace('.png', '.txt'))
        with open(label_path, "w") as f:
            f.write("0 0.5 0.5 0.5 0.5\n")  # Dummy bounding box (centered)

# For Validation Images
for img in os.listdir(os.path.join(path, "images/val")):
    if img.endswith(".jpg") or img.endswith(".png"):
        label_path = os.path.join(label_val, img.replace('.jpg', '.txt').replace('.png', '.txt'))
        with open(label_path, "w") as f:
            f.write("0 0.5 0.5 0.5 0.5\n")  # Dummy bounding box (centered)

print("✅ Dummy YOLO labels created for training!")


FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/images/train'

In [26]:
!ls /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3


dataset  dataset.yaml  labels  processed_images  scenes.csv


In [27]:
!ls /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/processed_images


In [28]:
!ls /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/processed_images


In [29]:
                            !find /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3 -type f \( -iname "*.jpg" -o -iname "*.png" -o -iname "*.jpeg" \)


Streaming output truncated to the last 5000 lines.
/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec1/sec1_frame_001481.png
/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec1/sec1_frame_001579.png
/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec1/sec1_frame_002807.png
/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec1/sec1_frame_001435.png
/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec1/sec1_frame_003559.png
/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec1/sec1_frame_002462.png
/root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset/sec1/sec1_frame_002595.png
/root/.cache/kaggl

In [30]:
dataset_yaml = """
path: {path}  # Dataset root path
train: dataset
val: dataset

names:
  0: car
  1: bike
  2: truck
"""

yaml_path = os.path.join(path, "dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(dataset_yaml.format(path=path))

print("✅ Dataset config file created at:", yaml_path)


✅ Dataset config file created at: /root/.cache/kagglehub/datasets/javiersanchezsoriano/traffic-images-captured-from-uavs/versions/3/dataset.yaml


In [4]:
!df -h


Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   37G   71G  35% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  775M  61% /usr/sbin/docker-init
tmpfs           6.4G   76K  6.4G   1% /var/colab
/dev/sda1        85G   66G   20G  78% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [3]:
model.train(
    data=yaml_path,
    epochs=20,
    imgsz=416,
    batch=8,
    name="traffic_detection_model"
)


NameError: name 'model' is not defined